# Test to fecth pdf with py

In [1]:
import fitz  # PyMuPDF
import json
from pprint import pprint
from collections import Counter
import pandas as pd


In [2]:
doc = fitz.open("test.pdf")
doc

Document('test.pdf')

In [2]:
doc_template = fitz.open("template_3.pdf")
doc_template

Document('template_3.pdf')

In [ ]:
"""all_spans = []

for page_num, page in enumerate(doc_template, 1):
    blocks = page.get_text("dict")["blocks"]
    for block in blocks:
        if "lines" not in block:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                all_spans.append({
                    #"text": span["text"].strip(),
                    "size": round(span["size"], 1),
                    #"font": span["font"],
                    #"flags": span["flags"],
                    #"page": page_num
                })

font_counter = Counter([s["size"] for s in all_spans])
most_common = font_counter.most_common()

print(most_common)"""


[(12.0, 51), (7.5, 4), (22.5, 3), (18.0, 2), (30.0, 1)]


In [4]:
df_font_count = pd.DataFrame(most_common, columns=['font_size', 'count']).sort_values(by='font_size', ascending=False)
df_font_count


NameError: name 'most_common' is not defined

In [3]:
nbr_pages = len(doc_template)
total_character = 0

data_json = {
    'nbr_page': nbr_pages,
    'nbr_character': total_character,
    'content': []
}

for page_num, page in enumerate(doc_template, 1):
    blocks = page.get_text("dict")["blocks"]

    # Characters length
    texte = page.get_text()
    total_character += len(texte)

    for block in blocks:
        if "lines" not in block:
            continue

        for line in block['lines']:
            for span in line['spans']:
                data_json['content'].append({
                    'page': page_num,
                    'text': span["text"].strip(),
                    'size': round(span["size"], 1),
                    'font': span["font"],
                    'flags': span["flags"],
                    #'important': "Bold" in span["font"] or span["flags"] == 20 #If font-bold, text is important
                })

data_json['nbr_character'] = total_character

data_json

{'nbr_page': 2,
 'nbr_character': 3749,
 'content': [{'page': 1,
   'text': 'Titre du cours',
   'size': 30.0,
   'font': 'Inter-Bold',
   'flags': 20},
  {'page': 1,
   'text': 'Chapitre 1',
   'size': 22.5,
   'font': 'Inter-SemiBold',
   'flags': 20},
  {'page': 1,
   'text': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque consequat leo',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4},
  {'page': 1,
   'text': 'id ornare suscipit. Curabitur hendrerit libero non elit sodales cursus. Nunc',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4},
  {'page': 1,
   'text': 'viverra accumsan mi, a dictum nulla posuere eu. Fusce ullamcorper vitae enim',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4},
  {'page': 1,
   'text': 'eget placerat. Phasellus blandit sit amet mi a dictum. Quisque lacinia massa est,',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4},
  {'page': 1,
   'text': 'ut placerat eros egestas non. Donec dui turpis, 

In [4]:
font_sizes = Counter()

for span in data_json['content']:
    font_sizes[span["size"]] += 1

sizes_sorted = sorted(font_sizes, reverse=True)

print(sizes_sorted)

[30.0, 22.5, 18.0, 12.0, 7.5]


In [5]:
size_to_level = {size: i + 1 for i, size in enumerate(sizes_sorted)}
size_to_level

{30.0: 1, 22.5: 2, 18.0: 3, 12.0: 4, 7.5: 5}

In [6]:
for span in data_json['content']:
    span["level"] = size_to_level[span["size"]]

data_json

{'nbr_page': 2,
 'nbr_character': 3749,
 'content': [{'page': 1,
   'text': 'Titre du cours',
   'size': 30.0,
   'font': 'Inter-Bold',
   'flags': 20,
   'level': 1},
  {'page': 1,
   'text': 'Chapitre 1',
   'size': 22.5,
   'font': 'Inter-SemiBold',
   'flags': 20,
   'level': 2},
  {'page': 1,
   'text': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque consequat leo',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4,
   'level': 4},
  {'page': 1,
   'text': 'id ornare suscipit. Curabitur hendrerit libero non elit sodales cursus. Nunc',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4,
   'level': 4},
  {'page': 1,
   'text': 'viverra accumsan mi, a dictum nulla posuere eu. Fusce ullamcorper vitae enim',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4,
   'level': 4},
  {'page': 1,
   'text': 'eget placerat. Phasellus blandit sit amet mi a dictum. Quisque lacinia massa est,',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4,


In [11]:
for span in data_json['content'][:10]:
    print(f"{span['text']} — size: {span['size']} → level {span['level']}")

data_json

Titre du cours — size: 30.0 → level 1
Chapitre 1 — size: 22.5 → level 2
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque consequat leo — size: 12.0 → level 4
id ornare suscipit. Curabitur hendrerit libero non elit sodales cursus. Nunc — size: 12.0 → level 4
viverra accumsan mi, a dictum nulla posuere eu. Fusce ullamcorper vitae enim — size: 12.0 → level 4
eget placerat. Phasellus blandit sit amet mi a dictum. Quisque lacinia massa est, — size: 12.0 → level 4
ut placerat eros egestas non. Donec dui turpis, tincidunt eu sagittis non, porta — size: 12.0 → level 4
non est. Phasellus rutrum vel metus nec efficitur. Duis maximus purus et nisl — size: 12.0 → level 4
egestas, ut pulvinar lorem venenatis. Cras condimentum tellus et magna ornare — size: 12.0 → level 4
dictum. Phasellus id condimentum diam. — size: 12.0 → level 4


{'nbr_page': 2,
 'nbr_character': 3749,
 'content': [{'page': 1,
   'text': 'Titre du cours',
   'size': 30.0,
   'font': 'Inter-Bold',
   'flags': 20,
   'level': 1},
  {'page': 1,
   'text': 'Chapitre 1',
   'size': 22.5,
   'font': 'Inter-SemiBold',
   'flags': 20,
   'level': 2},
  {'page': 1,
   'text': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque consequat leo',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4,
   'level': 4},
  {'page': 1,
   'text': 'id ornare suscipit. Curabitur hendrerit libero non elit sodales cursus. Nunc',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4,
   'level': 4},
  {'page': 1,
   'text': 'viverra accumsan mi, a dictum nulla posuere eu. Fusce ullamcorper vitae enim',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4,
   'level': 4},
  {'page': 1,
   'text': 'eget placerat. Phasellus blandit sit amet mi a dictum. Quisque lacinia massa est,',
   'size': 12.0,
   'font': 'Inter-Regular',
   'flags': 4,


In [7]:
structured_json = {
    'nbr_page': data_json['nbr_page'],
    'nbr_character': data_json['nbr_character'],
    'content': []
}

structure = structured_json['content']
stack = []

for i in data_json["content"]:
    node = {
        "text": i["text"],
        "level": i["level"],
        "children": []
    }

    if not stack:
        structure.append(node)
        stack.append(node)
        continue

    if node["level"] > stack[-1]["level"]:
        stack[-1]["children"].append(node)
        stack.append(node)

    elif node["level"] == stack[-1]["level"]:
        stack[-1]["text"] += " " + node["text"]

    else:
        while stack and node["level"] <= stack[-1]["level"]:
            stack.pop()
        if stack:
            stack[-1]["children"].append(node)
        else:
            structure.append(node)
        stack.append(node)

# Affichage
print('Structured JSON')
structured_json


Structured JSON


{'nbr_page': 2,
 'nbr_character': 3749,
 'content': [{'text': 'Titre du cours',
   'level': 1,
   'children': [{'text': 'Chapitre 1',
     'level': 2,
     'children': [{'text': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque consequat leo id ornare suscipit. Curabitur hendrerit libero non elit sodales cursus. Nunc viverra accumsan mi, a dictum nulla posuere eu. Fusce ullamcorper vitae enim eget placerat. Phasellus blandit sit amet mi a dictum. Quisque lacinia massa est, ut placerat eros egestas non. Donec dui turpis, tincidunt eu sagittis non, porta non est. Phasellus rutrum vel metus nec efficitur. Duis maximus purus et nisl egestas, ut pulvinar lorem venenatis. Cras condimentum tellus et magna ornare dictum. Phasellus id condimentum diam. Nam fermentum ultricies aliquam. Sed congue aliquet eros, vel rutrum risus lobortis a. Nunc congue blandit urna ac tincidunt. Etiam faucibus vel ligula nec egestas. Donec rutrum finibus ultrices. Etiam consequat finibus accumsan. 

In [8]:
def remove_empty_children(node):
    # Si le noeud a des enfants, on les traite récursivement
    if "children" in node:
        node["children"] = [remove_empty_children(child) for child in node["children"]]
        if not node["children"]:
            del node["children"]
    return node

structured_json['content'] = [remove_empty_children(node) for node in structured_json['content']]

In [9]:
structured_json

{'nbr_page': 2,
 'nbr_character': 3749,
 'content': [{'text': 'Titre du cours',
   'level': 1,
   'children': [{'text': 'Chapitre 1',
     'level': 2,
     'children': [{'text': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque consequat leo id ornare suscipit. Curabitur hendrerit libero non elit sodales cursus. Nunc viverra accumsan mi, a dictum nulla posuere eu. Fusce ullamcorper vitae enim eget placerat. Phasellus blandit sit amet mi a dictum. Quisque lacinia massa est, ut placerat eros egestas non. Donec dui turpis, tincidunt eu sagittis non, porta non est. Phasellus rutrum vel metus nec efficitur. Duis maximus purus et nisl egestas, ut pulvinar lorem venenatis. Cras condimentum tellus et magna ornare dictum. Phasellus id condimentum diam. Nam fermentum ultricies aliquam. Sed congue aliquet eros, vel rutrum risus lobortis a. Nunc congue blandit urna ac tincidunt. Etiam faucibus vel ligula nec egestas. Donec rutrum finibus ultrices. Etiam consequat finibus accumsan. 

In [ ]:
# EN FAIRE UNE FONCTION POUR POUVOIR L4UTILISER SUR CHAQUE PDF LU